In [1]:
import matchms
from matchms.importing import load_from_mgf
from cudams.utils import download

spectra = list(load_from_mgf(download('pesticides.mgf')))

In [18]:
large_file = download('GNPS_LIBRARY.mgf')

100%|███████████████████████████████████████| 128M/128M [00:00<00:00, 61.1GB/s]


In [19]:
from typing import Generator, TextIO, Union
from pyteomics.mgf import MGF
from joblib import Parallel, delayed
from matchms.importing.parsing_utils import parse_spectrum_dict
from matchms.Spectrum import Spectrum
from tqdm import tqdm

jobs = []
metadata_harmonization = True
with MGF(large_file, convert_arrays=1) as reader:
    reader = zip(reader, range(100_000))
    spectra = Parallel(-1)(
        delayed(parse_spectrum_dict)(spectrum=pyteomics_spectrum, metadata_harmonization=metadata_harmonization) 
        for pyteomics_spectrum,_ in tqdm(reader)
    )

14244it [00:12, 1154.52it/s]


In [28]:
from matchms.filtering import default_filters, normalize_intensities, reduce_to_number_of_peaks

def parse_spectrum(spectrum):
    spectrum = default_filters(spectrum)
    spectrum = normalize_intensities(spectrum)
    spectrum = reduce_to_number_of_peaks(spectrum, n_max=1024) # NOTICE! We cut off smaller peaks, after the largest `max_peaks` - this is required for the GPU
    return spectrum

spectra_proc = Parallel(-1)(
    delayed(parse_spectrum)(spe) for spe in spectra
)
spectra_proc = [sp for sp in spectra_proc if sp is not None]

2024-03-25 15:27:03,295:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-03-25 15:27:03,301:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-03-25 15:27:03,394:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-03-25 15:27:03,768:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-03-25 15:27:03,771:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-03-25 15:27:03,778:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-03-25 15:27:03,785:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-03-25 15:27:03,812:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-03-25 15:27:03,827:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H

In [32]:
import pickle
from pathlib import Path

Path('data').mkdir(exist_ok=True)
file = Path('data/GNPS-LIBRARY-default-filter-nmax-1024.pickle')
file.write_bytes(pickle.dumps(spectra_proc))
spectra_load = pickle.loads(file.read_bytes())

In [33]:
!gh release upload --clobber samples-0.1 {file}

Successfully uploaded 1 asset to samples-0.1
